In [ ]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle


# import training_checkpoints as note

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle

# note.depends_loaded()

rotations = 3
bins = 6

data = epfl_data.Data(rotations,bins)

# print(data.samples[0])
# print(data.samples[1])
print(data.samples[2][0])
# print(data.samples[3])

training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts_1 = []
training_tgts_2 = []
training_tgts_3 = []
training_tgts_f1 = []
training_tgts_f2 = []
training_tgts_f3 = []
train_angles = []
train_angles_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts_1.append(data.samples[2][d][0])
        training_tgts_2.append(data.samples[2][d][1])
        training_tgts_3.append(data.samples[2][d][2])
        training_images_f[d] = f_image
        training_tgts_f1.append(data.samples[2][d][0])
        training_tgts_f2.append(data.samples[2][d][1])
        training_tgts_f3.append(data.samples[2][d][2])
        
        train_angles.append(np.deg2rad(data.samples[1][d]))
        train_angles_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))

print(training_images.shape)
train = np.append(training_images, training_images_f,axis=0)
print(train.shape)
# shuffled_training = shuffle(train)
for i in range(len(training_tgts_1)):
    training_tgts_1.append(training_tgts_f1[i])
    training_tgts_2.append(training_tgts_f2[i])
    training_tgts_3.append(training_tgts_f3[i])
# training_tgts.append(training_tgts_f)
for i in range(len(train_angles)):
    train_angles.append(train_angles_f[i])
print('done')

In [ ]:
print(data.samples[2][0])
print(data.samples[2][0][0])
p=np.zeros((2,2))
pp=np.ones((2,2))
print(np.append(p,pp,axis=0))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras.losses import categorical_crossentropy as cross

In [ ]:
rotations = 3

In [ ]:
hog_input = Input(shape=(11016,), name='input')
layers = Dense(1000, input_dim=11016, activation='relu')(hog_input)
layers = Dropout(.5)(layers)
layers = Dense(100, activation='relu')(layers)
layers = Dropout(.5)(layers)
layers = Dense(10, activation='relu')(layers)
layers = Dropout(.5)(layers)
outputs = []
for r in range(rotations):
    outputs.append(Dense(bins, activation='softmax', name='output{}'.format(r))(layers))



In [ ]:
print(outputs)

In [ ]:
model = Model(inputs=hog_input, outputs=outputs)
model.compile(optimizer='adam', loss=cross, metrics=['accuracy'])
model.fit(train, 
          [np.asarray(training_tgts_1), np.asarray(training_tgts_2), np.asarray(training_tgts_3)], 
          batch_size=32, epochs=10, verbose=1)

In [ ]:
second_round = model.predict(train)
print(second_round)

In [ ]:
tester = 555
print(second_round[0][tester])
print(second_round[1][tester])
print("")
print(training_tgts_1[tester])
print(training_tgts_2[tester])
print(len(second_round[0]))
print(len(second_round[0][3]))

print(training_tgts_1)

In [ ]:
second_array = np.asarray(second_round)
print(second_array.shape)
second_array = np.moveaxis(second_array, 1,0)
print(second_array.shape)
print(len(second_array))
print(len(second_array[1]))
print(len(second_array[:][0]))
print(second_array[1][0])
print(second_array[2][0])
print(second_array[0][1])
print(second_array[0][0])
flat_second = second_array.reshape((-1, 18))
print("now: {}".format(flat_second.shape))
# print(flat_second.shape)
# # print(flat_second)
# print('new')
# print(flat_second[0])
# print('new')
# print(flat_second[1])
# print('new')
# print(flat_second[2])
# print('new')
# print(flat_second[3])
# print('new')
# print(flat_second[4])

# probabilities=[]
# new_input = []
# for outs in range(len(second_array.shape[1])):
#     for i in range(rotations):
#         probabilities.append(list(second_round[i][outs]))
#     new_input.append([item for sublist in probabilities for item in sublist])
# print(new_input)

# test = np.asarray(([[1,1,1],[2,3,4,5],[6,6,6,6,6]]))
# print(test)

In [ ]:
# print(second_array[:][0][0])
print(flat_second.shape)
print(flat_second[0])

In [ ]:
rot_input = Input(shape=(1,6,), dtype='float32', name='1_input')
rot_input2 = Input(shape=(1,6,), dtype='float32', name='2_input')
rot_input3 = Input(shape=(1,6,), dtype='float32', name='3_input')

layer = keras.layers.concatenate([rot_input, rot_input2, rot_input3])
layer = Dense(200, activation='relu')(layer)
layer = Dropout(.5)(layer)
layer = Dense(100, activation='relu')(layer)
layer = Dropout(.5)(layer)
layer = Dense(10, activation='relu')(layer)
layer = Dropout(.5)(layer)
output = Dense(1)(layer)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

In [ ]:
rot1 = tf.placeholder(tf.float32, [None, rotations,bins])
rot2 = tf.placeholder(tf.float32, [None, rotations,bins])
rot3 = tf.placeholder(tf.float32, [None, rotations,bins])
labelz = tf.placeholder(tf.float32, [None])
flatten1 = flatten(rot1)
flatten2 = flatten(rot2)
flatten3 = flatten(rot3)
W1 = tf.Variable(tf.zeros([2654, 100]))
B1 = tf.Variable(tf.zeros([100]))
Act1 = tf.nn.relu(tf.multiply(tf.concat(rot1,rot2,rot3),W1)+B1)
mse = tf.metics.mean_squared_error(labels=labelz, predictions = Act1)
optimizer = tf.trian.AdamOptimizer(.0001).minimize(mse)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(20):
        sess.run(optimizer, feed_dict={rot1:second_array[0][0], rot2:second_array[0][1], rot3:second_array[0][2]})


In [ ]:
model_two = Model(inputs=[rot_input, rot_input2, rot_input3], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.fit([second_array[0][:],second_array[1][:],second_array[2][:]], 
          train_angles, 
          batch_size=32, epochs=10, verbose=1)
#shuffle the array?

In [ ]:
score = model_two.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)
# note.all_complete()

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)

In [ ]:
# model_two = Sequential()
# model_two.add(Dense(200), input_dim=rotations*bins, activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(100), activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(10), activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(1), activation='sigmoid')

In [ ]:
model_two.fit(second, np.asarray(training_tgts), batch_size=32, epochs=20, verbose=1)

score = model.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)
# note.all_complete()

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)